In [ ]:
import pandas as pd
import pickle
import os
import nbimporter

import fill_in_blanks



In [ ]:
x =2 
x

In [ ]:
all_csv_to_dfs = []
directory = "./scada_data/30okt"

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        full_path = os.path.join(directory, filename)
        print(full_path)
        df = pd.read_csv(os.path.join(directory, filename),sep=";")
        all_csv_to_dfs.append(df)

        continue 
    else: 
        continue

In [ ]:
len(all_csv_to_dfs)

In [ ]:
for df in all_csv_to_dfs:
    print(df.TimeStamp[0])

### Make sure all dataframes have the same length

In [ ]:
# Show one dataframe
# Splitting dataframes from date: 25-09-2017 13:20:00

all_csv_to_dfs = fill_in_blanks.slice_df(all_csv_to_dfs)

In [ ]:
import numpy as np
# Cutting away rows after date: "20-10-2019 13:20:00"
# All dataframes should now have the same shape[1].
i = 0
for dataframe in all_csv_to_dfs:

    location = np.where(dataframe["TimeStamp"].str.contains("20-10-2019 13:20:00"))
    print(location)
    split_on_index = location[0][0]
    print(split_on_index)

    # Splitting
    df1 = dataframe.iloc[:split_on_index, :]
    df2 = dataframe.iloc[split_on_index:, :]

    # Reseting index
    df1 = df1.reset_index(drop=True)
    all_csv_to_dfs[i] = df1
    i+=1

In [ ]:
all_csv_to_dfs[0].iloc[:,0]

In [ ]:
# If a csv contains more than one:

import pandas as pd
import numpy as np
import gc
# Function returns a list of dfs for each datafram
def parce_csv_with_multiple_wt(df):

  scada_dfs1 = []
  for i in range(0,25):
      if (i < 9):
          wt = ("BESS-WTG0{0}".format(i+1))
      else:
          wt = ("BESS-WTG{0}".format(i+1))

      ''' 
       Check which column names contain the string wt.
       Fro example, BESS-WT02XXXXX, is a paramenter for turbine 02
       and is filtered into a seperate dataframe:
      '''
      # Take out the TimeStamp column, since it is being filtered away:
      time_stamp_column = df.iloc[:,0]

      filtered = df[df.columns[df.columns.to_series().str.contains(wt)]]
      filtered.insert(loc=0, column='TimeStamp', value=time_stamp_column)

      # Add it to the return list
      if (filtered.shape[1] > 0):
        print("FILTERED", filtered.shape)

        scada_dfs1.append(filtered)
        print("Appended...")
  del df
  gc.collect()
    
  return scada_dfs1

In [ ]:
finished_list = []
for dataframe in all_csv_to_dfs:
  more_than_one = (dataframe.shape[1] > 35)

  if (more_than_one):
    print("more_than_one", more_than_one)
    
    # Split it into more
    parced_list = parce_csv_with_multiple_wt(dataframe)
    
    for parced in parced_list:
      print("parced.shape[1]: ", parced.shape[1])
      if (parced.shape[1] == 35):
        finished_list.append(parced)
  else:
    # just add it to the list if it has all the features we need
    if (dataframe.shape[1] == 35):
      finished_list.append(dataframe)



In [ ]:
print(len(finished_list))

In [ ]:
# Make a new array with unique dataframes

def check_in_in_list(df,list1):
    boolean = "false"
    for l in list1:
        if(df.columns.equals(l.columns)):
            boolean = "true"
    return boolean
            

def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if (check_in_in_list(x,unique_list) == "false"):
            unique_list.append(x) 
    return unique_list

unique_finished = unique(finished_list)


In [ ]:
# The unique dataframes:

print(len(unique_finished))
# for u in unique_finished:
unique_finished[0].head()

## Saving the dataframes to folder

In [ ]:
#path = "./DataFromBazefield/scada_data_bessaker/"
path = '/Volumes/Uten navn/scada_data_bessaker/'
# Sorting so that we have df from 01 -> 24.
sorted_list = sorted(unique_finished, key=lambda x: x.columns[1].split("-")[1], reverse=False)


for dataframe in sorted_list:
    windturbine_id = dataframe.columns[1].split("-")[1]
    print(windturbine_id)
    
    # SAVE IT: Uncomment
    dataframe.to_csv(path + windturbine_id + ".gz", compression='gzip')

    #dataframe.to_pickle(path + windturbine_id + ".csv")


In [ ]:
# Can actually call the functions in fill_in_blanks here